In [ ]:
# touchpad
from machine import TouchPad, Pin

pad = TouchPad(Pin(12))

Thumbslide

In [ ]:
# slide code

from machine import Pin, ADC
import time

pinx = 39
piny = 34

x = ADC(Pin(pinx))
x.atten(ADC.ATTN_11DB)    # set 11dB input attentuation (voltage range roughly 0.0v - 3.6v)
x.width(ADC.WIDTH_9BIT)   # set 9 bit return values (returned range 0-511)
y = ADC(Pin(piny))
y.atten(ADC.ATTN_11DB)    # set 11dB input attentuation (voltage range roughly 0.0v - 3.6v)
y.width(ADC.WIDTH_9BIT)   # set 9 bit return values (returned range 0-511)

while True:
    print(x.read(), y.read())
    time.sleep(0.1)

Environmental sensor BME280

In [ ]:
from machine import I2C, Pin

I2C_SCL = 27
I2C_SDA = 32

i2c = I2C(scl=Pin(I2C_SCL), sda=Pin(I2C_SDA))
i2c.scan()

from bme280 import BME280
BME280_I2CADDR = 0x76 # 118

bme = BME280(i2c=i2c, address=BME280_I2CADDR)

(t, p, h) = bme.read_compensated_data()

print(t)

Motion sensor MPU-9250

In [ ]:
from machine import I2C, Pin

I2C_SCL = 27
I2C_SDA = 32
# 0x68, 104
i2c = I2C(scl=Pin(I2C_SCL), sda=Pin(I2C_SDA))

import mpu9250
mpu = mpu9250.MPU9250(i2c)
mpu.magnetic


GPIO Extender MCP23017

In [ ]:
import mcp

I2C_SCL = 27
I2C_SDA = 32

io = mcp.MCP23017(address=0x20, gpioScl=I2C_SCL, gpioSda=I2C_SDA) # 32

# controls some output pins
'''
outPins = list(range(10,16))
nextVals = {}
for pinNum in outPins:
    io.setup(pinNum, mcp.OUT)
    nextVals[pinNum] = True
io.output_pins(nextVals)
'''
# monitors and prints some input pins
inPins = list(range(0,16))
for pinNum in inPins:
    io.setup(pinNum, mcp.IN)
    io.pullup(pinNum, True)
while True:
    print(io.input_pins(inPins))

MicroSD card reader

In [ ]:
from machine import Pin, SPI

spi = SPI(2, 14500000, miso=Pin(19), mosi=Pin(23), sck=Pin(18))

import os
from sdcard import SDCard

sd = SDCard(spi, cs=Pin(14))

os.mount(sd, '/sd')
os.listdir('/')

with open('/sd/test.txt', 'w') as f:
    f.write("Hello World!")

with open('/sd/test.txt', 'r') as f:
    print(f.read())


In [ ]:
# APA102 LEDS

from machine import Pin

pin_clk = 13
pin_data = 4

clk = Pin(pin_clk, Pin.OUT)
data = Pin(pin_data, Pin.OUT)

clk.value(0)
data.value(0)

def write_led(dat):
    for i in range(32):
        data.value(dat & 1)
        clk.value(1)
        clk.value(0)
        dat = dat >> 1

def write_color(r,g,b,gl):
    data.value(1)
    for i in range(3):
        clk.value(1)
        clk.value(0)

    for i in range(5):
        data.value(gl & (1 << (4-i)))
        clk.value(1)
        clk.value(0)

    for c in [r, g, b]:
        for i in range(8):
            data.value(c & (1 << (7-i)))
            clk.value(1)
            clk.value(0)

import time
for i in range(2000):
    write_led(0)
    write_color(i, i+144, i+1, 10)
    write_color(i, i, i+34, 10)
    write_color(i+50, i, i, 10)
    write_color(i, i+200, 0, 10)
    write_led(~0)
    time.sleep(0.01)


Microphone

- https://github.com/loboris/MicroPython_ESP32_psRAM_LoBo/wiki/adc
- https://docs.micropython.org/en/latest/esp32/quickref.html#adc-analog-to-digital-conversion

In [ ]:
from machine import Pin, ADC
import time

pin_mic = 35

mic = ADC(Pin(pin_mic))
mic.atten(ADC.ATTN_0DB)
mic.width(ADC.WIDTH_12BIT) 
while True:
    value = mic.read()
    val_8bit = int(value*(256/4096))
    dac.write(val_8bit)


Speaker

In [ ]:
# speaker - loboris

from machine import DAC, Pin

dac = DAC(26)
sh = Pin(2, Pin.OUT, Pin.PULL_UP) # old pin=4
sh.value(1)
t = DAC.TRIANGLE
f = 1000
dac.waveform(f, type=t, scale=0)

In [ ]:
# speaker - official
from machine import DAC, Pin
import math
import time

sh = Pin(2, Pin.OUT, Pin.PULL_UP)
sh.value(1)

dac = DAC(Pin(26))

# create a buffer containing a sine-wave
buf = bytearray(100)
for i in range(len(buf)):
    buf[i] = 128 + int(127 * math.sin(2 * math.pi * i / len(buf)))

bl = len(buf)
for i in range(9999999):
    dac.write(buf[i % bl])

In [ ]:
# speaker + microphone (loboris)
from array import array

data = array('B', [0]*16000)
freq = 8000

# SPEAKER
from machine import DAC, Pin

dac = DAC(26)
sh = Pin(2, Pin.OUT, Pin.PULL_UP) # old pin=4
sh.value(1)

# Micophone
from machine import Pin, ADC
import time

pin_mic = 35

mic = ADC(Pin(pin_mic))
mic.atten(ADC.ATTN_0DB)
mic.width(ADC.WIDTH_12BIT) 

dac.stopwave()
mic.collect(freq, readmv=False, data=data, wait=True)
mic.stopcollect()

dac.write_buffer(data, freq, mode=DAC.CIRCULAR, wait=True)

Display

In [ ]:
# display - official
from ili934xnew import ILI9341, color565
from machine import SPI, Pin

spi = SPI(
    2,
    baudrate=40000000,
    miso=Pin(19),
    mosi=Pin(23),
    sck=Pin(18))
display = ILI9341(spi,
    cs=Pin(0),
    dc=Pin(15),
    rst=Pin(5))
display.erase()
display.set_pos(0,0)
display.width = 240
display.height = 320

display.fill_rectangle(0, 0, 239, 319, color565(255,0,0))

In [ ]:
display.write("yo man sup dogg ok OK jeje")

In [ ]:
# loboris display
# https://github.com/loboris/MicroPython_ESP32_psRAM_LoBo/wiki/display

import machine, display, time, _thread, math

tft = display.TFT()
tft.init(tft.ILI9341, width=240, height=320, miso=19,mosi=23,clk=18,cs=0,dc=15, bgr=True)  # SR EXTERNAL
tft.image(0,0,'cat.jpg')

In [ ]:
# Display touch i2c

address = 0x38

# TODO - write a driver